## TriContent Pump Controls
This notebook uses the PyCont module (https://github.com/croningp/pycont) in order to control the TriContinent Pumps

More functions can be added to this notebook if necessary as the library used has a wide range of selection to pick from.


### Importing the necessary modules

In [1]:
import pycont.controller as cont
import time
import logging
import serial #in case we close the port in each function

### Loading the configuration file
The cell below enables the logger and configures the pump. The user might want to check if the configureation JSON file in the right directory - this will vary depending on where you clone the repository. To avoid any errors in the main file, the pump gets initialized and gets disconnected in each function, for general testing it can be initialized in the cell blow by uncommenting the initalizer.

In [2]:
logging.basicConfig(level=logging.INFO)
setup_config_file = r"C:\Users\Blackr\Documents\CCUS\MAPs\Pump_Config.json" # check your system

controller = cont.MultiPumpController.from_configfile(setup_config_file)

#initialize the pump - smart initialize is to avoid reinitializing, and resetting the pump back to zero position
#controller.smart_initialize()

### Filling the electrolyte chamber
The following function allows the pump to pump a certai volume from the specified valve.
The valves for the syringe pump we use is as follows (as listed under pump_protocol.py):
- "I" : valve input
- "O" : valve output
- "B" : valve bypass
- "E" : extra valve

Note that for a three way pump (which will be used for the electrolyte chamber set up) the valves are I, E, O listed closkvise starting from the inlet on the left.

 The volumes indicated are in mL. 

In [3]:
def fill_chamber(volume):
    """
    This function moves the pumps the desired volume in mL and pumps it to the electrolyte chamber.
    it pumps the liquid from the stock KHCO3 solution (valve "I") and outputs it to "O"
    returns only after the pump is done moving
    """
    controller.smart_initialize()

    #for i in range(1):
    #testing to run this for once, now it keeps injecting and ejecting forever if not interrupted

    if controller.pumps["water"].is_volume_pumpable(volume) and controller.pumps["water"].is_idle():
        controller.pumps['water'].transfer(volume_in_ml=volume, from_valve = "I", to_valve = "O")
        
    else:
        print("The pump can't pump this volume since it exceeds the available volume.")
        return False
    time.sleep(2)

    #the volume can be asked as an iquiry to the user 
    #if necessary or hard coded to the script
    if __name__ == "__main__":
        fill_chamber(volume)

fill_chamber(2.4) #testing

ControllerRepeatedError: Repeated Error from pump water

### Emptying the electrolyte chamber
The following function empties the electrolyte chamber by pumping the liquid from the "O" valve and delivering it to the "E" valve, to the waste beaker.

In [ ]:
def empty_chamber(volume:float):
    """
    Empties the electrolyte chamber by a desired volume. 
    
    wait attribute is set to True to ensure that the function
    returns only after the pump is done moving
    """
    controller.pumps["water"].smart_initialize()

    while True:

        if not controller.pumps["water"].is_volume_pumpable(volume) and controller.pumps["water"].is_idle():
            print("The pump can't pump this volume since it exceeds the available volume.")
            return False           
        else:
            controller.pumps['water'].transfer(volume_in_ml=volume, from_valve = "O", to_valve = "I")
   
        break
    
    if __name__ == "__main__":
        empty_chamber(volume)

empty_chamber(2.4)

#??? can there be a loop that iterates to pump volumes higher than the syringe? 
#    ans: yes but we have a syringe big enough for our needs
#should this be in the top file?

### Withdrawing and pumping liquid from the same valve
The function below withdraws liquid from the valve that the pump is currently placed in and delivers it to the same place.

In [ ]:
def pump_volume(volume:float):
  """
  This function moves the pump to the desired volume in mL. from the valve the pumps is at.
  wait attribute is set to True to ensure that the function
  returns only after the pump is done moving
  """
    
  
  controller.pumps["water"].pump(volume, wait = True)
  print(str('pumped liquid from ' + controller.pumps["water"].get_valve_position()))
  time.sleep(2)
  controller.pumps["water"].deliver(volume,wait = True) 
  print('delivered liquid to ' + controller.pumps["water"].get_valve_position())
  time.sleep(2)
    

pump_volume(volume = 5.0)

### Sample JSON Setup file 

#### Configuration file for one chemical
Below is a cell you can make edits as necessary on the chemical used with the syringe pump. You can make the changes the and run the cell which should update the configuratoin file attached to this repo, which is used for the commands. You might have to make edits in the code as the code calls chemical(s) listed in the list.

In [ ]:
#testing with water
import json

config_dict = {
    "io":
    {
        "port": "COM7",    #check your system
        "baudrate": 9600,
        "timeout": 1
    },
    "default":
    {
        "volume": 5,
        "micro_step_mode": 2,
        "top_velocity": 5000,
        "initialize_valve_position": "I"
    },
    "groups":
    {
        "chemicals": ["KHCO3 1M"]
    },
    "pumps":
    {
        "water":
        {
            "switch": "5"
        }
    }
}


jsonfile = json.dumps(config_dict, indent=4)

with open("sampleconfiguration.json", "w") as outfile:
    outfile.write(jsonfile)

#### Configuration file for multiple chemicals

In [ ]:
#sample configuration file with multiple chemicals - we will be dealing with CO2 and KHCO3 only
{"default": 
   {"volume": 5,
    "micro_step_mode": 2,
    "top_velocity": 24000,
    "initialize_valve_position": "I"},

 "hubs": [
    {"io": {"port": "COM7",
            "baudrate": 9600,
            "timeout": 1},
     "pumps": {"acetone": {"switch": "0"},
               "water": {"switch": "1", "top_velocity": 12000}
               }
    },
    {"io": {"port": "COM7",
            "baudrate": 9600,
            "timeout": 1},
     "pumps": {"pentanol": {"switch": "0"}}
    }
  ],
  "groups": {
    "chemicals": ["acetone", "water"],
    "oils": ["pentanol"]
  }
}  

In [ ]:
pycont.controller.C3000Controller(pump_io: ,
    name: 'TCP',
    address: 'COM7',
    total_volume: 3.0,
    micro_step_mode: int = 2,
    top_velocity: int = 6000,
    initialize_valve_position: str = 'I')

#tbh I don't really remember why I added this but I will keep it just in case

#### Commands that might be useful in the future

In [ ]:
controller.pumps["water"].get_raw_valve_position() #tells you what valve you are at

In [ ]:
#controller.pumps["water"].go_to_volume(2.0)
#controller.pumps["water"].pump(2.0) #this only allows you to specify which vlve you want to pump the liquid from where the other oen goes to volume not carin about the valve
#controller.pumps["water"].deliver(2.0)
controller.pumps["water"].get_volume()#tells how much volume there is in the pump
controller.pumps["water"].is_volume_pumpable(5.0)

### Robert Added Simple Use Case Below

Initialize Libraries

In [1]:
import pycont.controller as cont
import time
import logging
import serial #in case we close the port in each function

Load the configuration file and instantiate an instance of the controller, calling in the config file

In [2]:
#RB Added
logging.basicConfig(level=logging.INFO)
setup_config_file = r"C:\Users\Blackr\Documents\CCUS\MAPs\Pump_Config.json" # check your system

controller = cont.MultiPumpController.from_configfile(setup_config_file)

#initialize the pump - smart initialize is to avoid reinitializing, and resetting the pump back to zero position
#controller.smart_initialize()

Initialize the pump - you should see it move to the zero position

In [3]:
controller.smart_initialize()

Basic operation below which is good for our needs. Inputs are total volume, liquid inlet, and liquid outlet

I believe 'I' = 9 o'clock, 'O' = 3 o'clock, and 'E' = 12 o'clock

In [4]:
controller.pumps['liquid'].transfer(1, 'I', 'O') 

In [7]:
controller.pumps['liquid'].transfer(2, 'E', 'O') 